In [1]:
import sys
sys.path.append('../../dnn/')
from model.nets.fragilityaux import CNNFragility
from model.nets.ieegcnn import iEEGCNN
from model.nets.ieegseq import iEEGSeq

import model.train
from model.train import traincnn, trainseq


import processing.util as util

import time
import numpy as np
# np.random.seed(1234)
import math as m
import os

# import DNN frameworks
import tensorflow as tf
import keras

# import high level optimizers, models and layers
from keras.optimizers import SGD
from keras.preprocessing import sequence
from keras.layers.embeddings import Embedding

# utility for datasets and training
# from keras.utils.training_utils import multi_gpu_model
from keras.datasets import imdb
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
# imports tensorflow
# from keras import backend as K

from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

%load_ext autoreload
%autoreload 2

/Users/adam2392/Documents/dnn-unsupervised/.venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imsize=30    # the size of the PCA that you perform on the rest of the fragility map
n_colors=1
num_classes=2
num_timesteps=150
DROPOUT=True
numwins=500

cnn = CNNFragility(numwins=numwins, 
                   imsize=imsize,
                  n_colors=n_colors, 
                  num_classes=num_classes, 
                  num_timesteps=num_timesteps,
                  DROPOUT=DROPOUT)
cnn.buildmodel()
cnn.summaryinfo()
print(cnn.model.input_shape)

# SVG(model_to_dot(cnn.model).create(prog='dot', format='svg'))
print(cnn.model.summary())

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
{'DROPOUT': True,
 'filteringsize': (3,),
 'imsize': 30,
 'numchans': 1,
 'numclasses': 2,
 'numfilters': 32,
 'numlayers': (4, 2, 1),
 'poolingsize': (2,),
 'w_init': None}
[(None, 500, 1), (None, 30, 500, 1)]
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 500, 1)       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 30, 500, 1)   0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 498, 32)      128         input_1[0][0]                    
_____________

In [4]:
print(cnn.model1d.input_shape)
print(cnn.model1d.output_shape)
print(cnn.model1d.input)
print(cnn.model1d.output)
print(cnn.model1d.summary())

(None, 500, 1)
(None, 7552)
Tensor("input_3:0", shape=(?, 500, 1), dtype=float32)
Tensor("flatten_3/Reshape:0", shape=(?, ?), dtype=float32)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 500, 1)            0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 498, 32)           128       
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 496, 32)           3104      
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 494, 32)           3104      
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 492, 32)           3104      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 246, 32)           0         
_

In [23]:
print(cnn.model2d.input_shape)
print(cnn.model2d.summary())

(None, 32, 32, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        (None, 32, 32, 1)         0         
_________________________________________________________________
conv2d_71 (Conv2D)           (None, 30, 30, 32)        320       
_________________________________________________________________
conv2d_72 (Conv2D)           (None, 28, 28, 32)        9248      
_________________________________________________________________
conv2d_73 (Conv2D)           (None, 26, 26, 32)        9248      
_________________________________________________________________
conv2d_74 (Conv2D)           (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_75 (Conv2D)           (None, 10, 10, 64)        184

# Testing Training Procedure for Fragility Auxiliary Input

**To Do:**
* Create a robust pipeline that is not dependent on how data is formatted.
    * data should be formatted as:
    - include, or don't include seizure period
    - length of window
    - length of window before seizure
    - length of window after seizure
    - how to pad
    - if seizure onset/offset times are not available
    - include data that is not success as well (sometimes they may capture certain areas of true epileptogenicity)
* How does one deal with these instances and handle the data if it is "noisy"

Ideas:
* Loss function that is dependent on the Engel score
* Loss function that is create a noisy instance of where onset/offset times occur

In [3]:
from model.train import traincnnaux
from model.train.fragaux.processdata import SplitData

In [11]:
n_colors=1
num_classes=2       # dimension of output predictions
DROPOUT=True
pcsize = 40         # the size of the PCA that you perform on the rest of the fragility map
numwins = 500       # the dimension of time to use in windows

listofpats = [
    'jh105',
    'pt1', 'pt2', 'pt3',
    'pt8', 'pt11', 'pt13', 'pt15'
    'pt16', 'pt17',
    'la01',
    'la02', 'la03',
    'la05', 'la07',
    'ummc002', 'ummc003',
    'ummc004', 'ummc005',
    'ummc006', 
#     'ummc008',
#     'id003', 'id004', 'id005', 
#     'id008', 'id010', 'id013'
#     'ummc001', 'ummc007', 'ummc009'
]

rawdatadir = '/Volumes/ADAM LI/pydata/converted/'
datadir = '/Volumes/ADAM LI/pydata/output/monopolar/pert/'
datamunger = SplitData(pcsize, numwins, rawdatadir)
datamunger.loaddirofdata(datadir, listofpats)

print("We have %i datasets" % len(datamunger.datafilepaths))

ignoring  la01_inter_pertmodel.npz
ignoring  la01_inter_2_pertmodel.npz
ignoring  la02_inter_pertmodel.npz
ignoring  la03_inter_pertmodel.npz
ignoring  la04_ictal_pertmodel.npz
ignoring  la04_inter_pertmodel.npz
ignoring  la05_inter_pertmodel.npz
ignoring  la06_ictal_pertmodel.npz
ignoring  la06_inter_pertmodel.npz
ignoring  la07_inter_pertmodel.npz
ignoring  la08_ictal_pertmodel.npz
ignoring  la08_inter_pertmodel.npz
ignoring  la09_ictal_pertmodel.npz
ignoring  la09_inter_pertmodel.npz
ignoring  la10_ictal_pertmodel.npz
ignoring  la10_inter_pertmodel.npz
ignoring  la11_ictal_pertmodel.npz
ignoring  la11_inter_pertmodel.npz
ignoring  la12_ictal_pertmodel.npz
ignoring  la13_ictal_pertmodel.npz
ignoring  la13_inter_pertmodel.npz
ignoring  la15_ictal_pertmodel.npz
ignoring  la15_inter_pertmodel.npz
ignoring  la16_ictal_pertmodel.npz
ignoring  la16_inter_pertmodel.npz
ignoring  la17_ictal_pertmodel.npz
ignoring  pt1aslp1_pertmodel.npz
ignoring  pt6sz3_pertmodel.npz
ignoring  pt6sz4_pertmod

In [12]:
print(datamunger.datafilepaths[0])
print(numwins)

/Volumes/ADAM LI/pydata/output/monopolar/pert/la01_ictal/la01_ictal_pertmodel.npz
500


In [13]:
datamunger.formatdata()

type error in birth date probably, so just setting to nan

/Volumes/ADAM LI/pydata/output/monopolar/pert/la01_ictal/la01_ictal_pertmodel.npz


type error in birth date probably, so just setting to nan

/Volumes/ADAM LI/pydata/output/monopolar/pert/la02_ictal/la02_ictal_pertmodel.npz


type error in birth date probably, so just setting to nan

...

/Volumes/ADAM LI/pydata/output/monopolar/pert/ummc006_sz1/ummc006_sz1_pertmodel.npz


type error in birth date probably, so just setting to nan

/Volumes/ADAM LI/pydata/output/monopolar/pert/ummc006_sz2/ummc006_sz2_pertmodel.npz


type error in birth date probably, so just setting to nan

/Volumes/ADAM LI/pydata/output/monopolar/pert/ummc006_sz3/ummc006_sz3_pertmodel.npz
/Volumes/ADAM LI/pydata/output/monopolar/pert/pt17sz1/pt17sz1_pertmodel.npz
/Volumes/ADAM LI/pydata/output/monopolar/pert/pt17sz2/pt17sz2_pertmodel.npz
/Volumes/ADAM LI/pydata/output/monopolar/pert/pt17sz3/pt17sz3_pertmodel.npz


In [20]:
print(len(datamunger.ylabels))
print(len(datamunger.aux_data))
print(len(datamunger.main_data))

6079
6079
6079


In [24]:
# setup training scheme for data
datamunger.trainingscheme(scheme='rand')

In [35]:
pcsize = 40
numwins = 500
NUM_EPOCHS = 100
AUGMENT = True
batch_size = 32

tempdatadir = 'test'

trainer = traincnnaux.TrainFragAux(cnn.model, numwins, batch_size, NUM_EPOCHS, AUGMENT)

In [38]:
class_weight = datamunger.class_weight
Xmain_train = datamunger.Xmain_train
Xaux_train = datamunger.Xaux_train
y_train = datamunger.y_train
Xmain_test = datamunger.Xmain_test
Xaux_test = datamunger.Xaux_test
y_test = datamunger.y_test

trainer.loadformatteddata(Xmain_train, Xmain_test, Xaux_train, Xaux_test, 
                          y_train, y_test, class_weight)
trainer.configure(tempdatadir)
trainer.loadgenerator()